# Imports

In [1]:
import gpt_wrapper
from gpt_wrapper.chat import Chat
%load_ext autoreload
%autoreload 2

gpt_wrapper.api_key = "a0e78a0b-9ef6-4d71-a054-75b743c03de7"

# Introduction

In [ ]:
chat1 = Chat.create("Test Chat")
chats = Chat.list()
chats = Chat.list(name="test")
chat1.to_dict()

In [ ]:
message = chat1.ask(content="Who won the FIFA championship in 2018?")
print(message)
message = chat1.ask(
    "Who was the best goalie in that match?",
    model_args={
        "temperature": 0.7, # default is 0.7
        "max_tokens": 60, # default is 100
        "top_p": 0.8, # default is 0.9
        "presence_penalty": 0.1, # default is 0.1
        "frequency_penalty": 0.1 # default is 0.1
     })
print(message)

In [ ]:
chat2 = Chat.create("Test Chat 2")
message = chat2.ask("Is earth a perfect sphere?")
print(message)
message = chat2.ask("Is earth a perfect sphere?", instruction="You are an assistant that speaks like Shakespeare.")
print(message)
print(message.to_dict())

Message object has the following fields available:
   - message_id: this message's unique ID
   - chat_id: the chat session ID the message belongs to
   - content: the message content
   - role: there are three options for role,
   - user: the user role belongs to you
   - assistant: the assistant role belongs to the ChatGPT
   - system: the system role belongs to the system-level instruction that you can pass with the instruction parameter in ask
   - created_at: the time the message was created
   - usage: number of tokens used to generate this message, only set for "assistant" messages
   - total_tokens: total input and output tokens the message has used
   - prompt_tokens: how many tokens your input has used
   - completion_tokens: how many tokens the ChatGPT model has generated
   - model_args: arguments passed to GPT API to produce this message, only set for "assistant" message


In [ ]:
messages = chat2.messages()
print([str(message) for message in messages])
# or
print([message.to_dict() for message in messages])

In [ ]:
message.to_dict()['role']

In [ ]:
Chat.budget()

# Testing strategies

In [32]:
import json
def get_data(path = "data/47.json"):
    with open(path) as f :
        data = json.load(f)
    return data

def save_data(data):
    path = "data/m1_submission_example.json"
    with open(path, 'w') as f :
        json.dump(data, f)

def ask_chat_gpt(chat, prompt, instruction):
    message = chat.ask(prompt,
                   instruction = instruction)
#     print(message)
    return message

def format_json(data_point, chat, message):
    data_point['chat_id'] = chat.to_dict()['chat_id']
    data_point['interaction'] = {
        'role' : message.to_dict()['role'],
        'content' : message.to_dict()['content']
    }
    return data_point

In [88]:

data = get_data()

### Method 0 : baseline
We only ask chat-GPT to answer the multiple choice question :

`prompt`: the question

`instruction` : 
- for MCQ : You are given multiple choice question with X possible solutions. Here is the question : Q. Here is the answers : [A1 ,A2, A3, A4]
- for others : nothing
    

__Obeservations__
- The answers do not follow a specific format 
- The possible solutions that are false are not developped whereas it would be interesting for a student to know why they are false
- For some answers, only the answer is given and there is no development
- For the mathematical question, there is no development

In [65]:
for i, data_point in enumerate(data[:10]) :
    print(f"##########  {i+1}  ##########")
    chat = Chat.create(f"chat #{i}")
    prompt = data_point['question']
    if 'choices' in data_point :
        answers = ' '.join([f"""{i+1}. "{answer}" """ for i, answer in enumerate(data_point['choices'])])
        instruction = f"""You are given multiple choice question with {len(data_point['choices'])} possible solutions. Here is the question : {data_point['question']}. Here is the answers : {answers} """
    else 
        instruction = ""
    print(data_point['question']+'\n')
    ask_chat_gpt(chat, prompt, instruction)
    print("#########################")

##########  1  ##########
Spatial localization in MRI primarily relies on...

Varying magnetic field across the patient.
#########################
##########  2  ##########
What is the main difference between PET and SPECT?

The main difference between PET and SPECT is that PET scan uses positron-emitting radioisotope instead of single-photon emitters, while SPECT uses single-photon-emitting radioisotopes. Therefore, the correct answer is option 4.
#########################
##########  3  ##########
How to distinguish different frequencies in a signal?

The correct answer is 3. "With Fourier transform". The Fourier transform is a mathematical technique that can be used to identify the frequency components of a signal. It works by decomposing a signal into its constituent frequencies, allowing us to identify and distinguish the different frequency components present in the signal. Option 2, "With Fourier series", is also related to frequency analysis, but it is used to represent periodi

## Method 1
Now we ask chat-GPT to develop each possible answers and say if they are true or false. We add the fact that we want the answer to be unique and that the answer must be true in any cases.

### For MCQ questions
`prompt`: There is a multiple choice question with X possible solutions. 
    Here is the question : Q. 
    Here are the answers : [A1 ,A2, A3, A4]

`instruction` : Develop each possible answers saying if they can be true or false and why it is the case. <br>
    Finally give the number of the final and unique answer that is true in any cases
### For open questions
`prompt`: the question

`instruction` : Answer to the question and say why it is your answer

__Obeservations__
- We get sometimes that multiple answers are true or that the answer can be true or false depending on what we suppose. We might add the information that we need no supposition more than what is written in the question
- Sometimes it says that no answers are true 
- We also do not handle the case where there are multiple right answers

In [71]:
for i, data_point in enumerate(data[9:14]) :
    chat = Chat.create(f"chat #{i}")
    if 'choices' in data_point :
        answers = ' '.join([f"""{i+1}. "{answer}" """ for i, answer in enumerate(data_point['choices'])])

        prompt = f"""There is a multiple choice question with {len(data_point['choices'])} possible solutions. 
        Here is the question : {data_point['question']}. 
        Here are the answers : {answers} """

        instruction = """Develop each possible solution saying if they can be true or false and why it is the case. 
        Finally give the number of the final and unique answer that is true in any cases."""
    else :
        prompt = f"""{data_point['question']} """

        instruction = """Answer to the question and say why it is your answer"""
    
    print(f"##########  {i+1}  ##########")
    print(data_point['question']+'\n')
    ask_chat_gpt(chat, prompt, instruction)
    print("\n")

##########  1  ##########
What is the goal of filterback projection?

1. "It helps to automatically remove artifacts and blurring" - False, filterback projection does not remove artifacts and blurring automatically, it helps in image reconstruction.
2. "It reconstructs the images from helical to axial orientation" - True, filterback projection is a common technique used to reconstruct a tomographic image from projection data obtained in a helical CT scan and to convert it into an axial image.
3. "It doubles the amount of scan data" - False, filterback projection does not double the amount of scan data, it uses the existing data to reconstruct the image.
4. "It records data to compute the dose used for the scan" - False, filterback projection does not record data to compute the dose used for the scan, it is a technique used for image reconstruction. 

Therefore, the final and unique answer that is true in any case is option 2: "It reconstructs the images from helical to axial orientatio

## Method 2
We add two things with this method
   1. We give a role to the model saying that it is a university teacher for Imaging course and that he will correct an homework he gave to its students
   2. Before answering, we ask him to state the necessary information needed 
This will be helpful because it will permit the model to think before answering. Morevover, the explaination given in the beginning will help the student understand better what is asked and what are the concepts necessary to answer the question.

### For MCQ questions
`prompt`: There is a multiple choice question with X possible solutions. 
    Here is the question : Q. 
    Here are the answers : [A1 ,A2, A3, A4]

`instruction` : You are a university teacher for Imaging course. You gave homework to your students and now correct it.<br>
        For each question, only suppose what is written in the question.<br>
        You first explain what is asked in the question.<br>
        Then you give definitions, theorems and lemmas that will be usefull to answer the question. <br>
        Next, develop each possible answers saying if it is true or false (not always true) and why it is the case. <br>
        Finally give the number of the final and unique answer that is true in any cases.
### For open questions
`prompt`: the question

`instruction` : You are a university teacher for Imaging course. You gave homework to your students and now correct it.<br>
        For each question, only suppose what is written in the question.<br>
        You first explain what is asked in the question.<br>
        Then you give definitions, theorems and lemmas that will be usefull to answer the question. <br>
        Finally, answer to the question and say why it is your answer.

__Obeservations__
- For questions where we ask what is the role of a technique, it says that no answer is true although it says that an answer is correct but because it is not the primary role of the technique, it says it is not a good answer.
- The format is different for every question. It could be good to have all the questions with the same format and to guide more the model in this perspective.



In [75]:
for i, data_point in enumerate(data[9:14]) :
    chat = Chat.create(f"chat #{i}")
    if 'choices' in data_point :
        answers = ' '.join([f"""{i+1}. "{answer}" """ for i, answer in enumerate(data_point['choices'])])

        prompt = f"""There is a multiple choice question with {len(data_point['choices'])} possible solutions. 
        Here is the question : {data_point['question']}. 
        Here are the answers : {answers}."""

        instruction = """You are a university teacher for Imaging course. You gave homework to your students and now correct it.
        For each question, only suppose what is written in the question.
        You first explain what is asked in the question.
        Then you give definitions, theorems and lemmas that will be usefull to answer the question. 
        Next, develop each possible answers saying if it is true or false (not always true) and why it is the case. 
        Finally give the number of the final and unique answer that is true in any cases."""
    else :
        prompt = f"""{data_point['question']} """

        instruction = """You are a university teacher for Imaging course. You gave homework to your students and now correct it.
        For each question, only suppose what is written in the question.
        You first explain what is asked in the question.
        Then you give definitions, theorems and lemmas that will be usefull to answer the question. 
        Finally, answer to the question and say why it is your answer"""
    
    print(f"##########  {i+1}  ##########")
    print(data_point['question']+'\n')
    ask_chat_gpt(chat, prompt, instruction)
    print("\n")

##########  1  ##########
What is the goal of filterback projection?

The question is asking about the goal of filterback projection.

Definitions/Theorems/Lemmas:
- Filterback projection is a reconstruction technique used in computed tomography (CT).
- It involves applying a filter to the measured projection data before back-projecting it to reconstruct the image.

Answer:
1. "It helps to automatically remove artifacts and blurring" - False. While filterback projection can help reduce artifacts and blurring, this is not its primary goal.
2. "It reconstructs the images from helical to axial orientation" - False. This is not the goal of filterback projection.
3. "It doubles the amount of scan data" - False. Filterback projection does not increase the amount of scan data.
4. "It records data to compute the dose used for the scan" - False. Filterback projection is not involved in dose calculation.

The correct answer is not given among the options. The goal of filterback projection is to 

## Method 3
We add two things with this method
   1. We say that it should answer true if we cannot find a counter example that make the statement false so that we avoid the case where the answer proposed is not the best answer but it is still true.
   2. We give chat-GPT instruction about the structure so that it is easier to read and to compare
   3. We ask chat-GPT to not give the same example for every answers so that we can get more information why the answers are true or false.


### For MCQ questions
`prompt`: There is a multiple choice question with X possible solutions. 
    Here is the question : Q. 
    Here are the answers : [A1 ,A2, A3, A4]

`instruction` : You are a university teacher for Imaging course. You gave homework to your students and now correct it. <br>
For each question, only suppose what is written in the question.<br>
You first explain what is asked in the question.<br>
Then you give definitions, theorems and lemmas that will be usefull to answer the question. <br>
Next, develop each possible answers saying if it is true or false and why it is the case by giving an different argument for each answers. For each answer structure it by using the following pattern "Answer #x : \<the xth answer\> : \<True or False\>. The reason why you answered this" <br>
An answer is true if we can not find a counter example. An answer is false if we can find a counter example that show that the statement is false. If the answer is false provide a counter-example.<br>
Finally give the number of the final answer (the is unique except if in the question it is said that there might be multiple answers) that is true in any cases.
    
### For open questions
`prompt`: the question

`instruction` : You are a university teacher for Imaging course. You gave homework to your students and now correct it.<br>
For each question, only suppose what is written in the question.<br>
You first explain what is asked in the question.<br>
Then you give definitions, theorems and lemmas that will be usefull to answer the question. <br>
Finally, answer to the question and say why it is your answer

__Obeservations__
- It is hard to always have the same structure but because we will group the data of all the students, we will not have the same structure anyway. We we could do to improve it is to construct few-shots and to give it in the instruction so that it can structure the answer as in the examples.


In [124]:
for i, data_point in enumerate(data[55:]) :
    chat = Chat.create(f"chat #{55+i+1}")
    if 'choices' in data_point :
        answers = ' '.join([f"""{i+1}. "{answer}" """ for i, answer in enumerate(data_point['choices'])])

        prompt = f"""There is a multiple choice question with {len(data_point['choices'])} possible solutions. 
        Here is the question : {data_point['question']}. 
        Here are the answers : {answers}."""

        instruction = """You are a university teacher for Imaging course. You gave homework to your students and now correct it.
For each question, only suppose what is written in the question.
You first explain what is asked in the question.
Then you give definitions, theorems and lemmas that will be usefull to answer the question. 
Next, develop each possible answers saying if it is true or false and why it is the case by giving an different argument for each answers. For each answer structure it by using the following pattern "Answer #x : <the xth answer> : <True or False>. The reason why you answered this" 
An answer is true if we can not find a counter example. An answer is false if we can find a counter example that show that the statement is false. If the answer is false provide a counter-example.
Finally give the number of the final answer (the is unique except if in the question it is said that there might be multiple answers) that is true in any cases."""
    else :
        prompt = f"""{data_point['question']} """

        instruction = """You are a university teacher for Imaging course. You gave homework to your students and now correct it.
For each question, only suppose what is written in the question.
You first explain what is asked in the question.
Then you give definitions, theorems and lemmas that will be usefull to answer the question. 
Finally, answer to the question and say why it is your answer"""
    
    print(f"##########  {55+i+1}  ##########")
    print(data_point['question']+'\n')
    message = ask_chat_gpt(chat, prompt, instruction)
    messages.append(message)
    data_point = format_json(data_point, chat, message)
    print("\n")

##########  56  ##########
Note: Make sure you select all of the correct optionsâ€”there may be more than one!

The question is asking about the characteristics of a certain imaging technique. 

Answer 1: "Detects the relative increase of oxyhemoglobin with respect to deoxyhemoglobin in neuronal areas activated." This statement is true, and it is referring to functional magnetic resonance imaging (fMRI), which measures the blood oxygen level-dependent (BOLD) signal. When a particular area of the brain is activated, there is an increase in blood flow to that area, which results in a change in the ratio of oxyhemoglobin to deoxyhemoglobin. This change in the ratio is what is detected by fMRI.

Answer 2: "Is used primarily in structural imaging." This statement is false. Structural imaging techniques, such as magnetic resonance imaging (MRI) and computed tomography (CT), are used to provide information about the brain's anatomy. fMRI, on the other hand, is a functional imaging technique t

The question is asking for the percentage of energy transferred in a Compton interaction when the deflection angle is zero degrees. 

Compton scattering is an inelastic scattering process between a photon and an electron. In this process, the incident photon transfers a portion of its energy to the electron, causing it to recoil and creating a scattered photon with a longer wavelength than the incident photon.

The energy transfer in a Compton interaction can be calculated using the Compton formula:

Δλ = h/mc (1 - cosθ)

where Δλ is the change in wavelength of the scattered photon, h is Planck's constant, m is the mass of the electron, c is the speed of light, and θ is the deflection angle of the scattered photon.

When θ = 0 degrees, the Compton formula simplifies to:

Δλ = h/mc

The change in energy can be calculated using the formula:

ΔE = hc/λ - hc/λ'

where λ is the wavelength of the incident photon and λ' is the wavelength of the scattered photon.

Using the relationship betwee

The question is related to the process of Medical Imaging and how the data is obtained from the patient's body. The question asks to select all the possible answers from the given options.

To answer this question, we need to understand the process of Medical Imaging. Medical Imaging is a technique used to create visual representations of the interior of a body for clinical analysis and medical intervention. The process involves the use of different types of energy to create images of tissues, organs, and bones. X-rays, CT scans, MRI, and Ultrasound are some of the commonly used Imaging techniques.

Answer 1: "X-rays pass through the patient then fall onto special detectors." This answer is true. X-rays are a type of electromagnetic radiation that has high energy and can penetrate through the body. X-rays pass through the body and are absorbed by the detectors placed on the opposite side of the body. 

Answer 2: "The detectors measure the transmission values or attenuation values." Thi

The question is asking about the method used for X-ray detection in Emission Tomography. 

Definitions/Theorems/Lemmas: 
- Emission Tomography: a medical imaging technique that uses radioactive material to produce images of the inside of the body.
- X-ray detection: a process of measuring the intensity and other properties of X-rays.
- Beta camera: a device that uses a scintillator crystal to convert beta particles to visible light, which is then detected by a photomultiplier tube (PMT).
- PMT: a device that converts light into an electrical signal.
- Scintillation: a process in which a material emits light when it absorbs radiation.
- Amplification: a process of increasing the strength of a signal.

Answer #1: "A beta camera composed of PMT and scintillating crystal" : True. 
Explanation: Beta camera is one of the devices used in X-ray detection in Emission Tomography. It consists of a scintillating crystal that absorbs the X-rays and converts them into visible light. The visible ligh

The question is asking for the difference between \(T_2^*\) and \(T_2\). 

Definition: 
- \(T_2\) is the time constant for the decay of the transverse magnetization due to spin-spin interactions.
- \(T_2^*\) is the time constant for the decay of the transverse magnetization due to spin-spin interactions and field inhomogeneities.

Answer #1: "An extra 90° pulse" : False. This answer is talking about the difference between spin echo imaging and gradient echo imaging, not the difference between \(T_2^*\) and \(T_2\).

Answer #2: "Spin-lattice interactions" : False. Spin-lattice interactions are involved in both \(T_2\) and \(T_2^*\) relaxation processes, so this answer does not explain the difference between them.

Answer #3: "Dependence on the homogeneities of the main magnetic field" : True. The difference between \(T_2^*\) and \(T_2\) is due to the additional decay of the transverse magnetization caused by field inhomogeneities in \(T_2^*\). This effect is not present in \(T_2\), whic

The question is asking about the way Fourier transform analyzes signals. Fourier transform is a mathematical technique that decomposes a signal into its constituent frequencies. It is commonly used in signal processing and image analysis.

Answer 1: "into amplitudes and reported by their phase" - False. This is the description of a complex number in polar form, not how Fourier transform analyzes signals.

Answer 2: "into frequencies and reported by their amplitudes" - True. This is the correct way of describing the Fourier transform. It separates the signal into its frequency components and reports their amplitudes.

Answer 3: "into amplitudes and reported by their frequencies" - False. This answer is the opposite of the correct answer. Fourier transform separates the signal into frequencies, not amplitudes.

Answer 4: "into phases and reported by their amplitudes" - False. This answer is similar to answer 1 but with the roles of phase and amplitude switched. Fourier transform separate

The question is about the definition of a RF pulse. 

A RF pulse is a short duration of radio frequency radiation used to excite the nuclear spins in the sample being imaged in MRI (Magnetic Resonance Imaging). It is applied perpendicular to the magnetic field. 

Answer #1: "A radio wave applied for a small amount of time" : True. This is the correct definition of a RF pulse. 

Answer #2: "A magnetic field gradient" : False. A magnetic field gradient is a variation of the magnetic field strength along one direction. It is used to spatially encode the signal in MRI, but it is not a RF pulse. 

Answer #3: "A magnetic field oscillating at radiofrequency" : False. A magnetic field oscillating at radiofrequency is the result of a RF pulse, it is not the definition of a RF pulse. 

Therefore, the final and unique answer that is true is answer #1: "A radio wave applied for a small amount of time".


##########  93  ##########
The probability of a Compton interaction per unit mass is _________

The signal-to-noise ratio (SNR) is defined as the ratio of the signal intensity to the noise intensity. A higher SNR indicates a clearer image with less noise. 

To increase CT signal-to-noise ratio (SNR), we need to increase the signal while keeping the noise constant or reducing it. Here are some definitions and theorems that will be useful to answer the question:

- Matrix size: The matrix size refers to the number of pixels in the image. Increasing the matrix size will increase the spatial resolution of the image, but it will also increase the noise. The SNR may increase or decrease depending on the imaging parameters.
- Field of view (FOV): The field of view refers to the size of the area being imaged. Increasing the FOV will decrease the spatial resolution of the image, but it will also decrease the noise. The SNR may increase or decrease depending on the imaging parameters.
- mAs: The mAs (milliamperes per second) is a measure of X-ray tube current and exposure time. Increasing 

In [111]:
save_data(data)

In [13]:
# Adding the confidence to the data
with open('data/confidence_MD.text') as f :
    confidence = f.read().split('\n')
data = get_data(path = 'data/m1_submission_example.json')
for i, d in enumerate(data) :
    d['confidence'] = int(confidence[i])
    d['interaction'] = [d['interaction']]
save_data(data)

In [16]:
from check_submission_json import test 
test()

##################################################
Searching for json files...
['C:\\Users\\mathi\\Documents\\EPFL_Master\\MA4\\assignment-cs-522\\project-m1-chat-mma\\data\\47.json', 'C:\\Users\\mathi\\Documents\\EPFL_Master\\MA4\\assignment-cs-522\\project-m1-chat-mma\\data\\m1_submission_example.json']
Found 1 json files (excluding the given data package).
--------------------------------------------------
Verifying the format of:  C:\Users\mathi\Documents\EPFL_Master\MA4\assignment-cs-522\project-m1-chat-mma\data\m1_submission_example.json
Found 100 interaction entries in the json file C:\Users\mathi\Documents\EPFL_Master\MA4\assignment-cs-522\project-m1-chat-mma\data\m1_submission_example.json.
##################################################
Successfully passed format verification for demonstration json files.


## Chatgpt pour Paraphrasing detection

In [44]:
instruction_en = """You are given two sentences namely sentence1 and sentence2 and your goal is to determine whether this pair of sentences have the same meaning.
Two sentences are considered to have the same meaning if they convey the same core concept or idea and if they serve the same purpose or achieve the same communicative goal.
If you think sentence1 and sentence2 has the same meaning, output only the number "1". If they don't output only the number "0".

Here are a few independant examples :
With the input 
    sentence1 : The family moved to Camp Hill in 1972 , where he attended Trinity High School in Harrisburg , Pennsylvania ., 
    sentence2: In 1972 , the family moved to Camp Hill , where he visited the Trinity High School in Harrisburg , Pennsylvania .,
we want as ouput : 
    1  

With the input 
    sentence1 : Components of elastic potential systems store mechanical energy if they are deformed when forces are applied to the system ., 
    sentence2: Components of elastic potential systems store mechanical energy if they are deformed to the system when applied to forces .,
we want as ouput : 
    1  

With the input 
    sentence1 : In mathematical astronomy , his fame is due to the introduction of the astronomical globe , and his early contributions to understanding the movement of the planets ., 
    sentence2: His fame is due in mathematical astronomy to the introduction of the astronomical globe and to his early contributions to the understanding of the movement of the planets .,
we want as ouput : 
    1  

With the input 
    sentence1 : The Hudson County , New Jersey , also spelled as `` Macpelah Cemetery '' , or `` Macphelah Cemetery '' , is a cemetery in Machpelah Cemetery ., 
    sentence2: The Machpelah Cemetery , also spelled as '' Macpelah Cemetery `` or '' Macphelah Cemetery `` , is a graveyard in Hudson County , New Jersey .,
we want as ouput : 
    0  

With the input 
    sentence1 : Aamir Khan has agreed to play immediately after reading Mehra 's script in `` Rang De Basanti '' ., 
    sentence2: Mehra agreed to act in `` Rang De Basanti '' immediately after reading Aamir Khan 's script .,
we want as ouput : 
    0  
    """
instruction_fr = """
On vous donne deux phrases, à savoir la phrase1 et la phrase2, et votre objectif est de déterminer si cette paire de phrases a le même sens.
Deux phrases sont considérées comme ayant le même sens si elles véhiculent le même concept ou la même idée de base et si elles servent le même but ou atteignent le même objectif de communication.
Si vous pensez que la phrase1 et la phrase2 ont le même sens, ne donnez que le chiffre "1". Si ce n'est pas le cas, ne donnez que le chiffre "0".

Voici quelques exemples indépendants :
Avec l'input 
    phrase1 : La famille déménagea à Camp Hill en 1972, où il alla à la Trinity High School à Harrisburg, en Pennsylvanie., 
    phrase2: En 1972, la famille déménagea à Camp Hill, où il visita la Trinity High School à Harrisburg, en Pennsylvanie.,
Repondez : 
    1  

Avec l'input 
    phrase1 : Les composants des systèmes à potentiel élastique stockent de l'énergie mécanique s'ils sont déformés quand des forces sont appliquées sur le système., 
    phrase2: Les composants des systèmes à potentiel élastique stockent l'énergie mécanique s'ils sont déformés par le système lorsqu'ils sont appliqués aux forces.,
Repondez : 
    1  

Avec l'input 
    phrase1 : En astronomie mathématique, sa renommée est due à l'introduction du globe astronomique et ses premières contributions à la compréhension du mouvement des planètes., 
    phrase2: Sa renommée est due à l'astronomie mathématique, à l'introduction du globe astronomique et à ses premières contributions à la compréhension du mouvement des planètes.,
Repondez : 
    1  

Avec l'input 
    phrase1 : Le compté de Hudson, dans le New Jersey, également orthographié "cimetière Macpelah", ou "cimetière Macphelah", est un cimetière dans le cimetière de Machpelah., 
    phrase2: Le cimetière Machpelah, également appelé "cimetière Macpelah" ou "cimetière Macphelah", est un cimetière du comté de Hudson, dans le New Jersey.,
Repondez : 
    0  

Avec l'input 
    phrase1 : Aamir Khan a de suite accepté de jouer après avoir lu le script de Mehra dans "Rang De Basanti"., 
    phrase2: Mehra a accepté de jouer dans "Rang de Basanti" juste après avoir lu le scénario d'Aamir Khan.,
Repondez : 
    0    
    """

In [49]:
from datasets import load_dataset

dataset_val = load_dataset('paws-x', 'en', split = 'validation')
dataset = load_dataset('paws-x', 'en', split = 'test')

Found cached dataset paws-x (C:/Users/mathi/.cache/huggingface/datasets/paws-x/en/1.1.0/37673404a6de6d0fa2574661e77940d10d3be3bf51bb4f08c8fa079fd56b9755)
Found cached dataset paws-x (C:/Users/mathi/.cache/huggingface/datasets/paws-x/en/1.1.0/37673404a6de6d0fa2574661e77940d10d3be3bf51bb4f08c8fa079fd56b9755)


In [50]:
dataset_val[0]

{'id': 4,
 'sentence1': 'From the merger of the Four Rivers Council and the Audubon Council , the Shawnee Trails Council was born .',
 'sentence2': 'Shawnee Trails Council was formed from the merger of the Four Rivers Council and the Audubon Council .',
 'label': 1}

In [51]:
for i in range(5,10):
    print(f"""Avec l'input 
    sentence1 : {dataset_val[i]['sentence1']}, 
    sentence2: {dataset_val[i]['sentence2']},
Repondez : 
    {dataset_val[i]['label']}  \n""")

Avec l'input 
    sentence1 : The family moved to Camp Hill in 1972 , where he attended Trinity High School in Harrisburg , Pennsylvania ., 
    sentence2: In 1972 , the family moved to Camp Hill , where he visited the Trinity High School in Harrisburg , Pennsylvania .,
Repondez : 
    1  

Avec l'input 
    sentence1 : Components of elastic potential systems store mechanical energy if they are deformed when forces are applied to the system ., 
    sentence2: Components of elastic potential systems store mechanical energy if they are deformed to the system when applied to forces .,
Repondez : 
    1  

Avec l'input 
    sentence1 : In mathematical astronomy , his fame is due to the introduction of the astronomical globe , and his early contributions to understanding the movement of the planets ., 
    sentence2: His fame is due in mathematical astronomy to the introduction of the astronomical globe and to his early contributions to the understanding of the movement of the planets .,
Re

In [54]:
output = []
for i, data_point in enumerate(dataset.select(list(range(200)))) :
    if i+1%50 == 0 :
        print(i+1)
#     print(i, data_point)
    chat = Chat.create(f"chat #{i}")

    prompt = f"""sentence1 is "{data_point['sentence1']}" and sentence2 is "{data_point['sentence2']}"""

    instruction = instruction_en
    message = ask_chat_gpt(chat, prompt, instruction)
    if '1' in message.content and '0' not in message.content :
        if data_point['label'] == 1 :
            output.append(1)
        else :
            output.append(0)
            
    elif '0' in message.content and '1' not in message.content :
        if data_point['label'] == 0 :
            output.append(1)
        else :
            output.append(0)

Server timeout.
Retrying in 10 seconds...


In [55]:
import numpy as np
np.mean(output) 
#fr avec consigne en fr 0.673469387755102
#fr avec consigne en anglais 0.5677083333333334
#en avec consigne en anglais 

0.5876288659793815